In [ ]:
import chromadb
from sentence_transformers import SentenceTransformer

In [ ]:
class Embedder():
    def __init__(self):
        self.model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')
    
    def __call__(self, input):
        input = self.model.encode(input).tolist()
        return input

In [ ]:
class ChromaDB():
    def __init__(self):
        self.embedding_model = Embedder()
        self.client = chromadb.PersistentClient(path='/media/space/ssd_1_tb_evo_sumsung/MishaHW/ChromaDB')
        self.collection = self.client.get_or_create_collection(name="colls", embedding_function=self.embedding_model)

    def add_collection(self, all_fragments, all_metadate):
        o_b = 0
        b = 20000
        ids = [str(i) for i in range(len(all_metadate))]
        while True:
            if b > len(all_fragments):
                break
            else:
                self.collection.add(
                    documents = all_fragments[o_b:b],
                    embeddings = self.embedding_model(all_fragments[o_b:b]),
                    metadatas = all_metadate[o_b:b],
                    ids=ids[o_b:b]
                )
                o_b = b
                b += 20000

        self.collection.add(
                    documents = all_fragments[o_b:b],
                    embeddings = self.embedding_model(all_fragments[o_b:]),
                    metadatas = all_metadate[o_b:],
                    ids=ids[o_b:]
                )
        print("Data was loaded")

    def search(self, text, count = 1):
        vector = self.embedding_model(text)
        result = self.collection.query(
            query_embeddings = vector,
            n_results = count,
            include=['distances','embeddings', 'documents', 'metadatas'],
        )
        return result
        

In [ ]:
from ctransformers import AutoModelForCausalLM
llm = AutoModelForCausalLM.from_pretrained("TheBloke/Mistral-7B-OpenOrca-GGUF", model_file="mistral-7b-openorca.Q4_K_M.gguf", model_type="mistral", gpu_layers=50)

In [ ]:
cdb = ChromaDB()

In [ ]:
from evaluate import load
bertscore = load("bertscore")

In [ ]:
def echo(message, history):
    result = cdb.search(message, 1)
    promt = f"Context: {result['documents'][0][0]}." + f"Question: {message}"
    answer = llm(promt)
    metric = bertscore.compute(predictions=result['documents'][0], references=[message], model_type="distilbert-base-uncased")
    return f"{answer}" +f"\n\nPrecision: {metric['precision'][0]}" + f"\nRecall: {metric['recall'][0]}"

In [ ]:
import gradio as gr

ex = ['How old are you?', 'What is your favorite color?', 'Where were you born?', 'Do you have any siblings?']
demo = gr.ChatInterface(fn=echo, examples=ex, title="Echo Bot")
demo.launch(share=True)